In [2]:
using Tullio
using StaticArrays
using EtherArrays
using KernelAbstractions
using oneAPI

In [3]:
@kernel function ker!(data)
    idx::Int = @index(Global)
    a = E2SquareMatrix{2}(idx, 1, data)
    b = E2SquareMatrix{2}(idx, 5, data)
    c = similar(a)
    c .= a
    # a .= a * b
    # a .= ein"ik,kj->ij"(c, b)
    @tullio a[i, j] := c[i, k] * b[k, j]
end

ker! (generic function with 4 methods)

In [4]:
data_cpu = randn(Float32, 3, 8);
data_gpu = oneArray(data_cpu)

3×8 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
  0.333476   1.7035     1.49651   …   0.368785   1.36923   -0.0771668
 -0.0709715  1.4379    -1.88446      -0.251039  -1.78635    0.516435
 -0.746565   0.638803  -0.671771     -2.54847    0.029178   0.771377

In [5]:
ker!(oneAPIBackend(), 3)(data_gpu, ndrange=(3,))
KernelAbstractions.synchronize(oneAPIBackend())

GPUCompiler.InvalidIRError: InvalidIRError: compiling MethodInstance for gpu_ker!(::KernelAbstractions.CompilerMetadata{KernelAbstractions.NDIteration.DynamicSize, KernelAbstractions.NDIteration.DynamicCheck, Nothing, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}, KernelAbstractions.NDIteration.NDRange{1, KernelAbstractions.NDIteration.DynamicSize, KernelAbstractions.NDIteration.StaticSize{(3,)}, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}, Nothing}}, ::oneDeviceMatrix{Float32, 1}) resulted in invalid LLVM IR
Reason: unsupported call to an unknown function (call to gpu_malloc)
Stacktrace:
 [1] malloc
   @ ~/.julia/packages/GPUCompiler/j4HFa/src/runtime.jl:85
 [2] gc_pool_alloc
   @ ~/.julia/packages/GPUCompiler/j4HFa/src/runtime.jl:116
 [3] multiple call sites
   @ unknown:0
Reason: unsupported dynamic function invocation (call to (ev::Tullio.Eval)(args...) @ Tullio ~/.julia/packages/Tullio/2zyFP/src/eval.jl:20)
Stacktrace:
 [1] macro expansion
   @ ~/.julia/packages/Tullio/2zyFP/src